In [ ]:
# # Colab library to upload files to notebook
# from google.colab import files

# # Install Kaggle library
# # Run if you don't have dataset
# # !pip install --upgrade --force-reinstall --no-deps kaggle
# !mv kaggle.json ../root/.kaggle/kaggle.json
# !chmod 600 ../root/.kaggle/kaggle.json
# !kaggle competitions download -c imaterialist-fashion-2019-FGVC6
# !unzip *.zip

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp /content/drive/MyDrive/Machine\ Learning\ Project/label_descriptions.json label_description.json
!cp /content/drive/MyDrive/Machine\ Learning\ Project/resized_train_images.zip resized_train_images.zip
!cp /content/drive/MyDrive/Machine\ Learning\ Project/train.csv.zip train.csv.zip

In [3]:
!unzip resized_train_images.zip
!unzip train.csv.zip

Streaming output truncated to the last 5000 lines.
  inflating: content/resized_train_images/5c816abf8e2ff6afa92ac421cb12b55d.jpg  
  inflating: content/resized_train_images/c916b657a3b7e44018ec032c7fc81122.jpg  
  inflating: content/resized_train_images/2ea1a3293ecf809ea348e6af5adaf97b.jpg  
  inflating: content/resized_train_images/30e10911bddcc0af779a5d5a4ddc4e97.jpg  
  inflating: content/resized_train_images/bbd891063bf9f98b50e648490113a0ae.jpg  
  inflating: content/resized_train_images/200e40b0fa2f1149739d282e656164d4.jpg  
  inflating: content/resized_train_images/62a6573ee501ce2542bef44aa9f2dabd.jpg  
  inflating: content/resized_train_images/9aa66fc02b0041db017443f069f2efb1.jpg  
  inflating: content/resized_train_images/c23f4ecbf6aa77d768c82e46b6ead719.jpg  
  inflating: content/resized_train_images/26a5c12b9bf964d15f9720a38244642f.jpg  
  inflating: content/resized_train_images/417f4be446ad163c8b5333d08a223526.jpg  
  inflating: content/resized_train_images/cfd391a92a418e17

In [4]:
!rm *.zip

In [5]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import json

import tensorflow as tf
from PIL import Image, ImageDraw
from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Input, Conv2DTranspose
from tensorflow.keras.layers import concatenate, Activation, MaxPooling2D

In [7]:
label_orig = json.load(open('label_description.json'))

In [8]:
train = pd.read_csv('train.csv')

In [9]:
train.columns

Index(['ImageId', 'EncodedPixels', 'Height', 'Width', 'ClassId'], dtype='object')

In [44]:
batch = 16
height = 256
width = 256
image_size = (batch,height,width,3)

In [45]:
images_index = list()
i = 0
max_len = len(train['ImageId'])
while(i < max_len):
    ImageId = train['ImageId'][i]
    images_index.append(i)
    while(i < max_len and train['ImageId'][i] == ImageId):
        i+=1

In [46]:
images_index_batches = list()
temp = list()
count = 0
for i in images_index:
    if(count == batch):
        images_index_batches.append(temp)
        count = 1
        temp = list()
        temp.append(i)
    else:
        count += 1
        temp.append(i)
images_index_batches.append(temp)

In [47]:
def rle_decode(mask_rle, shape=(768, 768)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T 

In [48]:
def imgshow(Index):
    plt.imshow(cv2.cvtColor(cv2.imread('content/resized_train_images/' + train['ImageId'][Index]), cv2.COLOR_BGR2RGB))

In [49]:
def mask_creator_visual(alpha):
    l = train['Height'][alpha]
    b = train['Width'][alpha]
    
    label = []
    
    mask_img_r = np.zeros((l,b), dtype=int)
    mask_img_g = np.zeros((l,b), dtype=int)
    mask_img_b = np.zeros((l,b), dtype=int)
    
    ImageId = train['ImageId'][alpha]
    
    while (aplha < len(train) and train['ImageId'][alpha] == ImageId):
        cat = int(train['ClassId'][alpha].split("_")[0])
        
        color = labels['categories'][cat]['color']
        
        label.append(labels['categories'][cat]['name'])
        
        rle_mask_pixels = rle_decode(train['EncodedPixels'][alpha], (b, l))
        
        rle_mask_pixels = (rle_mask_pixels == 1)

        mask_img_r[rle_mask_pixels] += color[0]
        mask_img_g[rle_mask_pixels] += color[1]
        mask_img_b[rle_mask_pixels] += color[2]
        
        alpha += 1
    
    mask = np.concatenate([mask_img_r.reshape((l,b,1)), mask_img_g.reshape((l,b,1)), mask_img_b.reshape((l,b,1))], axis = 2)
    
    return mask

In [50]:
def mask_creator_actual(alpha):
    l = train['Height'][alpha]
    b = train['Width'][alpha]
    
    ImageId = train['ImageId'][alpha]

    so = []
    
    while (alpha < len(train) and train['ImageId'][alpha] == ImageId):
        
        cat = int(train['ClassId'][alpha].split("_")[0])
        # print(alpha, cat)

        temp = rle_decode(train['EncodedPixels'][alpha], (b, l))
        
        temp = temp.reshape((1, l, b, 1))
        
        # print(temp.shape)
        temp1 = tf.image.resize(temp, [image_size[1], image_size[2]], method = 'area')

        temp1 = tf.reshape(temp1, [1, image_size[1], image_size[2], 1])
        # print(temp1.shape)
        
        so.append([cat, temp1])

        alpha += 1
    
    # mask_img = tf.image.resize(mask_img, [image_size[1], image_size[2]], method = 'area')

    so.sort(key = lambda x : x[0])

    count = 0
    for i in range(len(label_orig['categories'])):
        if (count < len(so) and so[count][0] == i):
            temp = so[count][1]
            count += 1
            while (count < len(so) and so[count][0] == i):
                temp = tf.add(temp, so[count][1])
                count += 1
            
        else:
            temp = tf.zeros([1,image_size[1],image_size[2],1])
        try:
            mask_img = concatenate([mask_img, temp], axis = 3)
        except:
            mask_img = temp
    return mask_img

In [51]:
def image_read(index):
    temp = tf.keras.preprocessing.image.load_img('content/resized_train_images/' + train['ImageId'][index])
    temp = tf.keras.preprocessing.image.img_to_array(temp)
    temp = tf.reshape(temp, [1,image_size[1], image_size[2], 3])
    return temp

In [52]:
def generator():
    for i in images_index_batches:
        # print(i)
        y_batch = mask_creator_actual(i[0])
        x_batch = image_read(i[0])
        for j in range(1, len(i)):
            y_batch =  concatenate( [ y_batch, mask_creator_actual(i[j]) ], axis =  0)
            x_batch = concatenate( [ x_batch,  image_read(i[j]) ], axis = 0 )
        yield (x_batch, y_batch)

In [53]:
def conv_up_block(input_tensor, input_concat, filters_no = 64, filter_size = 3):
    x = Conv2DTranspose(filters = filters_no, kernel_size = (filter_size, filter_size), strides = (2, 2), padding = 'same')(input_tensor)
    x = concatenate([input_concat, x], axis = 3)
    x = conv_block(x, filters = filters_no)
    return x

In [54]:
def conv_block(input_tensor, filters = 64, filter_size = 3):
    x = Conv2D(filters = filters, kernel_size = (filter_size, filter_size), activation='relu',
             padding = 'same', kernel_initializer='he_normal')(input_tensor)
    x = Conv2D(filters = filters, kernel_size = (filter_size, filter_size), activation='relu',
             padding = 'same', kernel_initializer='he_normal')(x)
    return x

In [55]:
def Unet():
    mpf = (2,2)
    
    x = Input(shape = (256,256,3,))
    
    # Contracting Path  
    # First Layer
    c1 = conv_block(x)
    m1 = MaxPooling2D(mpf)(c1)
    
    # Second Layer
    c2 = conv_block(m1, 128)
    m2 = MaxPooling2D(mpf)(c2)

    # Third Layer
    c3 = conv_block(m2, 256)
    m3 = MaxPooling2D(mpf)(c3)

    # 4th Layer
    c4 = conv_block(m3, 512)
    m4 = MaxPooling2D(mpf)(c4) 

    # 5th Layer
    c5 = conv_block(m4, 1024)
    m5 = MaxPooling2D(mpf)(c5)
    
    #6th Layer
    c6 = conv_block(m5, 2048)

    # Expanding Path
    # 7th Layer
    u7 = conv_up_block(c6, c5, 1024)

    # 8th Layer
    u8 = conv_up_block(u7, c4, 512)

    # 9th Layer
    u9 = conv_up_block(u8, c3, 256)

    # 10th Layer
    u10 = conv_up_block(u9, c2, 128)
    
    # 11th Layer
    u11 = conv_up_block(u10, c1, 64)

    output = Conv2D(len(label_orig['categories']), (1, 1), activation='sigmoid')(u11)
    
    model = Model(inputs = x, outputs = output)
    return model

In [56]:
unet = Unet()

In [57]:
unet.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_46 (Conv2D)              (None, 256, 256, 64) 1792        input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_47 (Conv2D)              (None, 256, 256, 64) 36928       conv2d_46[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_10 (MaxPooling2D) (None, 128, 128, 64) 0           conv2d_47[0][0]                  
_______________________________________________________________________________________

In [58]:
unet.compile(optimizer="Adam", loss = tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.MeanIoU(num_classes=len(label_orig['categories']))])

In [59]:
generator_object = generator()

In [60]:
unet.fit(generator_object, epochs=5, batch_size = batch)

Epoch 1/5
    137/Unknown - 835s 6s/step - loss: 1.0695 - mean_io_u_2: 0.2748

KeyboardInterrupt: ignored